<a href="https://colab.research.google.com/github/SumeyyeSuslu/regression/blob/main/sparkH2Odummyautoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu ornek spark uzerinde h2o ve sparkling water kullanarak autoencoder ve lineer regression algoritmasını kullanmaktadır.


**BU ornekte h20 kurulum ve init calisti, deep learning model olusturuldu dakat regression basarisiz. Autoencoder featurelari restructure ettigi icin x y z koordinatlarini da bozuyor. Ayrica rmse kullanimini yapamadik. Scriptin alti (print rmse den sonrasi)tamamen baska ornekten kopyalandi calistirilmadi.**


In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark
!pip install 'h2o<3.40.0.0'
!pip install h2o-pysparkling-3.3

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Reading package lists... Done
openjdk version "1.8.0_352"
OpenJDK Runtime Environment (bui

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
      #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZzZ1`1

In [ ]:
!export SPARK_LOCAL_IP='127.0.0.1'
!export MASTER='local[*]'

In [ ]:
#!tar xvf dummygeometryJSON.tar.gz 
from google.colab import drive
drive.mount('/content/drive')
!tar xvf /content/drive/MyDrive/dummygeometryJSON.tar.gz

Mounted at /content/drive
dummygeometry.json


In [ ]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
 
from pyspark.sql.functions import rand
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vector
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
import pyspark.sql.functions as F
import h2o
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator

from pysparkling import *

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()
sc = spark.sparkContext


In [ ]:
# Start H2O cluster
h2o.init()
h2oContext =  H2OContext.getOrCreate()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_352"; OpenJDK Runtime Environment (build 1.8.0_352-8u352-ga-1~20.04-b08); OpenJDK 64-Bit Server VM (build 25.352-b08, mixed mode)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5yf62zw7
  JVM stdout: /tmp/tmp5yf62zw7/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5yf62zw7/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_unknownUser_58vhxn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.813 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Connecting to H2O server at http://6356f88bbba4:54325 ... successful.


H2O_cluster_uptime:,19 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,sparkling-water-root_local-1676227128042
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,804 Mb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.38.0.4-1-3.3
 * H2O name: sparkling-water-root_local-1676227128042
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,172.28.0.12,54323)
  ------------------------

  Open H2O Flow in browser: http://6356f88bbba4:54325 (CMD + click in Mac OSX)

    


In [ ]:
df = spark.read.json('dummygeometry.json')
# Select the relevant columns
input_cols = ['mach','aoa','CoordinateX', 'CoordinateY','CoordinateZ','zone']
output_col = "Pressure Coefficient"
spark_df = df.select(input_cols + [output_col])
# Convert Spark DataFrame to H2OFrame
h2oFrame = h2oContext.asH2OFrame(spark_df)


In [ ]:
# Split the data into training and validation sets
train, valid = h2oFrame.split_frame(ratios=[0.8], seed=123)

# Define the autoencoder model
autoencoder = H2OAutoEncoderEstimator(
  activation="Tanh",
  hidden=[10, 5],
  epochs=100,
  seed=123
)

# Train the autoencoder model on the training set
autoencoder.train(
  x=train.names,
  training_frame=train,
  validation_frame=valid
)

# Make predictions on the validation set
predictions = autoencoder.predict(valid)

# Calculate RMSE

y_true = valid["Pressure Coefficient"]
y_pred = predictions["predict"]


# Stop H2O cluster
#h2o.shutdown()


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [ ]:
predictions.rmse()

AttributeError: ignored

In [ ]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='pressure', predictionCol='predict')
# Evaluate the performance of the autoencoder on the test data
rmse = evaluator.evaluate(predictions)
print("RMSE:", rmse)


AttributeError: ignored

In [ ]:
# Train the autoencoder on the training data
model = autoencoder.fit(train)

# Apply the autoencoder to the test data
encoded_test_data = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='pressure', predictionCol='latent_representation')

# Evaluate the performance of the autoencoder on the test data
rmse = evaluator.evaluate(encoded_test_data)
print("RMSE:", rmse)



In [ ]:
print(predictions)

  reconstr_mach    reconstr_aoa    reconstr_CoordinateX    reconstr_CoordinateY    reconstr_CoordinateZ    reconstr_zone    reconstr_Pressure Coefficient
       0.597561         9.96841                 11.8784                -1.6593                 0.139004        0.0761089                        -0.129142
       0.59667          9.96257                 11.867                 -1.67013                0.138275        0.0707741                        -0.129234
       0.5968           9.96381                 11.8863                -1.68741                0.142244        0.0699384                        -0.129682
       0.599889         9.98218                 12.0316                -1.52655                0.134491        0.0961401                        -0.125225
       0.598218         9.97144                 11.9567                -1.60099                0.141096        0.0949467                        -0.127685
       0.599966         9.9813                  12.0382                -1.48

In [ ]:
print(valid)

  mach    aoa    CoordinateX    CoordinateY    CoordinateZ    zone    Pressure Coefficient
   0.6     10        11.791        -1.66806       0.260443       0              -0.063004
   0.6     10        11.7686       -1.69392       0.26584        0               0.0328851
   0.6     10        11.8017       -1.71349       0.26528        0               0.0129717
   0.6     10        11.9673       -1.47686       0.256793       0              -0.311165
   0.6     10        11.8441       -1.56003       0.294707       0              -0.137517
   0.6     10        11.9211       -1.39          0.297938       0              -0.318486
   0.6     10        11.9872       -1.69439       0.260765       0              -0.0815489
   0.6     10        11.9671       -1.80103       0.265409       0              -0.0937527
   0.6     10        12.0598       -1.65057       0.299872       0              -0.276396
   0.6     10        12.106        -1.73743       0.267299       0              -0.0554022
[753

In [ ]:
from pyspark.ml.feature import Autoencoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

# Assemble the features into a single column
assembler = VectorAssembler(inputCols=['mach','aoa','CoordinateX', 'CoordinateY','CoordinateZ','zone'], outputCol="features")
data = assembler.transform(df)

# Initialize the autoencoder
autoencoder = Autoencoder(inputCol='features', outputCol='latent_representation', layers=[3, 2, 3])

# Split the data into training and test datasets
training_data, test_data = data.randomSplit([0.7, 0.3])

# Train the autoencoder on the training data
model = autoencoder.fit(training_data)

# Apply the autoencoder to the test data
encoded_test_data = model.transform(test_data)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='pressure', predictionCol='latent_representation')

# Evaluate the performance of the autoencoder on the test data
rmse = evaluator.evaluate(encoded_test_data)
print("RMSE:", rmse)

# Compute the RMSE between the original pressure values and the reconstructed pressure values



In [ ]:
df.summary().show()

In [ ]:
from pyspark.sql.functions import *

# filter rows where zone is 3 or 0 and mach is less than 0.8 and CoordinateZ is less than or equal to 0.26
filtered_df = df.filter((col("zone") == 3) | (col("zone") == 0) & (col("mach") < 0.8) & (col("CoordinateZ") <= 0.26))

In [ ]:
filtered_df.summary().show()

In [ ]:
# Split the dataset randomly into 80% for training and 20% for testing. Passing a seed for deterministic behavior
train_data, test_data = filtered_df.randomSplit([0.8, 0.2], seed = 0)

In [ ]:
vectorAssembler= VectorAssembler(inputCols=['mach','aoa','CoordinateX', 'CoordinateY','CoordinateZ' ],outputCol='features')

gbt = GBTRegressor(featuresCol='features',labelCol='Pressure')

In [ ]:

# Define a grid of hyperparameters to test:
#  - maxDepth: maximum depth of each decision tree 
#  - maxIter: iterations, or the total number of trees 
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 100])\
  .build()
 
# Define an evaluation metric.  The CrossValidator compares the true labels with predicted values for each combination of parameters, and calculates this value to determine the best model.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
 
# Declare the CrossValidator, which performs the model tuning.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)


In [ ]:

pipeline = Pipeline(stages=[vectorAssembler, cv])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_data)

# Make predictions.
predictions = model.transform(test_data)

# Select example rows to display.
predictions.select("prediction", "Pressure", "features").show(5)

# Select (prediction, true label) and compute test error
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[0]
print(rfModel)  # summary only